# **추출 체인(Extraction Chain) 구축하기**

이 노트북에서는 **채팅 모델(Chat Models)** 의 **도구 호출(Tool Calling)** 기능을 사용하여 **비정형 텍스트에서 구조화된 정보를 추출**하는 방법을 다룹니다. 또한 이 맥락에서 **Few-Shot 프롬프팅(Few-Shot Prompting)** 을 사용하여 성능을 개선하는 방법을 시연할 것입니다.

### 주요 내용
- Pydantic을 활용해 데이터 추출을 위한 스키마(Schema) 정의  
- LangChain의 Tool Calling 기능을 사용해 LLM이 구조화된 데이터를 반환하도록 설정  
- Few-Shot 프롬프팅(Few-Shot Prompting) 기법을 사용해 성능 향상  
- 다중 엔터티(Multiple Entity) 추출 지원 (여러 개의 인물 정보 추출 가능)

## **스키마 (The Schema)**  

먼저, 텍스트에서 어떤 정보를 추출할 것인지 **정의**해야 합니다.  

이를 위해 **Pydantic**을 사용하여 **개인 정보(personal information)** 를 추출하기 위한 예제 **스키마(schema)** 를 정의할 것입니다.

In [ ]:
# LangSmith 추적 설정 활성화

In [ ]:
class Person(BaseModel):
    # 이 문서 문자열은 LLM에 Person 스키마의 설명으로 전달되며, 추출 결과를 향상시키는 데 도움이 될 수 있습니다.
    # 참고 사항:
    # 1. 각 필드는 `optional`(선택적)입니다. → 이를 통해 모델이 해당 필드를 추출하지 않을 수도 있습니다!
    # 2. 각 필드는 `description`(설명)을 포함합니다. → 이 설명은 LLM이 활용하며,
    #    설명을 잘 작성하면 추출 결과를 더욱 향상시킬 수 있습니다.

## **스키마 정의의 두 가지 사례**

1. **속성(attributes)** 과 **스키마(schema)** 를 Pydantic으로 문서화  
   - 이 정보는 LLM에 전달되며, Pydantic 스키마를 통해 명확하게 정의되어 정보 추출의 품질을 개선하는 데 사용됩니다.
     
<pr></pr>

2. **LLM이 정보를 지어내지 않도록 합니다.**  
   - 각 속성에 `Optional`을 사용하여 LLM이 답을 모를 경우 `None`을 반환할 수 있도록 했습니다.
  
최상의 성능을 얻으려면 **스키마를 잘 문서화**하고, 텍스트에 추출할 정보가 없을 경우 모델이 결과를 **강제로 반환하지 않도록** 설정합니다.  


## **추출기 (The Extractor)**

이제 위에서 정의한 **스키마(schema)** 를 사용하여 **정보 추출기(Information Extractor)** 를 만들어 봅니다.

In [ ]:
# 사용자 정의 프롬프트 템플릿 정의
# 텍스트에서 정보를 추출하기 위한 명확한 지침과 추가 컨텍스트를 제공합니다.

**기능/도구 호출(Function/Tool Calling)** 을 지원하는 모델을 사용해야 합니다.

In [ ]:
# LLM에서 구조화된 출력을 생성하도록 스키마 바인딩

LLM은 생성 모델이므로, 센티미터로 제공된 신장의 정보를 미터로 정확하게 추출하는 등의 놀라운 작업을 수행할 수 있습니다! 또한 스키마에 정의된 이름, 머리색, 키 외의 다른 내용은 무시하고 답변을 생성합니다.


## 다중 엔터티

많은 경우, 단일 엔티티가 아닌 여러 엔티티를 추출해야 합니다. 이는 Pydantic에서 모델을 서로 중첩하여 쉽게 구현할 수 있습니다.

In [ ]:
class Data(BaseModel):

**여러 엔티티**를 추출할 수 있도록 스키마가 설계되면, 텍스트에 관련 정보가 없을 경우 **빈 리스트(empty list)** 를 반환하여 **아무런 엔티티도 추출하지 않을 수 있습니다.**  

이는 일반적으로 **좋은 설계**입니다! 이를 통해 모델이 해당 엔티티를 반드시 감지하도록 강제하지 않을 수 있습니다.  

## **Few-Shot Prompting**  

LLM 애플리케이션의 동작은 **Few-Shot 프롬프팅**을 사용하여 조정할 수 있습니다.  

**챗 모델(Chat Models)** 의 경우, 원하는 동작을 보여주는 **입력(input)** 과 **응답(response)** 메시지 쌍의 시퀀스로 구성될 수 있습니다.  

예를 들어, `user`와 `assistant` **메시지**가 번갈아 가며 나타나는 구조를 통해 `🦜` 기호의 의미를 전달할 수 있습니다.

## **구조화된 출력 (Structured Outputs)과 함수 호출**
 
챗봇과 같은 많은 애플리케이션에서는 모델이 사용자에게 **자연어로 직접 응답**해야 합니다. 그러나 경우에 따라 모델이 **구조화된 형식(structured format)** 으로 출력을 제공해야 할 필요가 있습니다.  

예를 들어, 모델의 출력을 **데이터베이스에 저장**해야 하는 상황에서 출력이 데이터베이스 **스키마(schema)**에 맞도록 보장해야 할 수 있습니다.  

이러한 필요성은 **구조화된 출력(Structured Output)** 개념을 부각시키며, 이를 통해 모델이 **특정 출력 구조**를 따르도록 지시할 수 있습니다.  

**핵심 포인트:**  
- 자연어 응답이 아닌 **구조화된 데이터**로 응답.  
- 데이터베이스, API 등과의 호환성을 보장.  
- 스키마를 따르는 일관된 형식으로 데이터 제공.

구조화된 출력은 특히 **데이터베이스 저장, API 연동, 데이터 처리** 등 다양한 응용 분야에서 중요한 역할을 합니다. 

In [ ]:
# 현재의 온도 가져오기
def get_temperature(latitude, longitude):
# 서울의 위도, 경도

In [ ]:
# 스키마 정의
# 스키마를 모델에 바인딩
# 사용자 입력
# 사용자 입력을 받아 스키마에 맞는 구조화된 출력을 생성하도록 모델 호출

In [ ]:
# structured_output을 함수 파라미터로 전달

In [ ]:
# 최종 응답을 생성하기 위한 LLM 프롬프트 설정
# LLM을 사용하여 최종 응답 생성
# 결과 출력